# Installing Libraries

In [1]:
!pip install streamlit

In [2]:
!pip install pyngrok

# Importing Libraries and Models

In [3]:
import streamlit as st  ## streamlit
import pandas as pd  ## for data manipulation
import pickle   ## For model loading 
import spacy  ## For NLP tasks 
import time

from PIL import Image   ## For image
from io import StringIO  ## for text input and output from the web app

In [5]:
from keras.models import load_model
# load model
model = load_model('/content/my_model.h5')
# summarize model.
model.summary()
# load dataset

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               51712     
_________________________________________________________________
dense_37 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_38 (Dense)             (None, 6)                 3078      
Total params: 317,446
Trainable params: 317,446
Non-trainable params: 0
_________________________________________________________________


In [7]:
# load the model from disk
# Load model from file
from sklearn.externals import joblib
classifier = joblib.load("/content/pickle_nb.pkl")
classifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

# Streamlit App

In [30]:
# defining the function which will make the prediction using the data which the user inputs
%%writefile app.py 
import streamlit as st
import numpy as np
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.externals import joblib

def welcome(): 
  return 'welcome all'

# defining the function which will make the prediction using  
# the data which the user inputs 
def prediction(tweet):
  from keras.models import load_model
  model = load_model('/content/sent_model.h5')
  classifier = joblib.load("/content/pickle_nb.pkl")
  # Making predictions 
  
  from keras.preprocessing.text import Tokenizer
  from keras.preprocessing.sequence import pad_sequences
  from sklearn.preprocessing import LabelEncoder
  import numpy as np
  #label_encoder = LabelEncoder()
  text=np.array([tweet])
  tk=Tokenizer(num_words=4000,lower=True,split=" ")
  tk.fit_on_texts(text)
  prediction = model.predict_classes(pad_sequences(tk.texts_to_sequences(text),
  maxlen=100))
  if prediction == 0:
    prediction = 'negative'
    return prediction
  elif prediction == 1:
    prediction = 'neutral'
    return prediction
  else:
    prediction = 'positive'
    return prediction
  text=np.array([tweet])
  tk=Tokenizer(num_words=4000,lower=True,split=" ")
  tk.fit_on_texts(text)
  prediction2 = classifier.predict_classes(pad_sequences(tk.texts_to_sequences(text),
  maxlen=100))
  if prediction2 == 0:
    prediction2 = 'negative'
    return prediction2
  elif prediction2 == 1:
    prediction2 = 'neutral'
    return prediction2
  else:
    prediction2 = 'positive'
    return prediction2
# this is the main function in which we define our webpage 
def main():
  import streamlit as st
   # giving the webpage a title 
  st.title("Tweet Prediction") 
  st.set_option('deprecation.showPyplotGlobalUse', False)
  st.sidebar.title("Info")

  st.markdown("## Input Tweet in English")
  st.sidebar.markdown("This app uses artificial neural networks to predict the sentiment of a tweet ")

    # front end elements of the web page 
  html_temp = """ 
  <div style ="background-color:purple;padding:13px"> 
  <h1 style ="color:black;text-align:center;">Streamlit Eagles Tweet App</h1> 
  </div> 
    """
    # display the front end aspect
  st.markdown(html_temp, unsafe_allow_html = True) 
    # following lines create boxes in which user can enter data required to make prediction 
  Tweet = st.text_input("Tweet")
  result =""
        
      # when 'Predict' is clicked, make the prediction and store it 
  if st.button("Predict"): 
    result = prediction(Tweet) 
  st.success('The sentiment of the tweet is {}'.format(result))
  # else:
  #   result = prediction2(Tweet) 
  # st.success('The aspect of the tweet is {}'.format(result))

data_path = ("/content/clean_health_data.csv")

@st.cache()
def load_data():
    data = pd.read_csv(data_path)
    data['tweet_created'] = pd.to_datetime(data['tweet_date'])
    return data


data = load_data()

st.sidebar.subheader("Show random tweets")
random_tweet = st.sidebar.radio('Select the Sentiment',('positive','negative','neutral'))

st.subheader("Here are some example of tweets according to your choice!")
st.markdown("1." + data.query("labels == @random_tweet")[['tweet']].sample(n=1).iat[0,0])


st.sidebar.markdown("### Choose Model")
select = st.sidebar.selectbox('Model Type',['ANN-Sentiment Model','Naive Bayes Model'])

st.sidebar.markdown("### Number of tweets")
select = st.sidebar.selectbox('Visualization Type',['BarGraph','PieChart'])

sentiment_count = data['labels'].value_counts()
sentiment_count = pd.DataFrame({'Sentiments':sentiment_count.index,'Tweets':sentiment_count.values})

def choose_model(model):
        st.markdown("### Choose Model")
        if select=='ANN-Sentiment Model':
            text=np.array([tweet])
            tk=Tokenizer(num_words=4000,lower=True,split=" ")
            tk.fit_on_texts(text)
            prediction = model.predict_classes(pad_sequences(tk.texts_to_sequences(text),
            maxlen=100))
            if prediction == 0:
              prediction = 'negative'
              return prediction
            elif prediction == 1:
              prediction = 'neutral'
              return prediction
            else:
              prediction = 'positive'
              return prediction 
        else:
            text=np.array([tweet])
            tk=Tokenizer(num_words=4000,lower=True,split=" ")
            tk.fit_on_texts(text)
            prediction2 = classifier.predict_classes(pad_sequences(tk.texts_to_sequences(text),
            maxlen=100))
            if prediction2 == 0:
              prediction2 = 'negative'
              return prediction2
            elif prediction2 == 1:
              prediction2 = 'neutral'
              return prediction2
            else:
              prediction2 = 'positive'
              return prediction2 

if st.sidebar.checkbox('Show',False,key='0'):
    st.markdown("### No. of tweets by sentiments ")
    if select=='BarGraph':
        fig = px.bar(sentiment_count,x='Sentiments',y='Tweets',color='Tweets',height=500)
        st.plotly_chart(fig)
    else:
        fig = px.pie(sentiment_count,values='Tweets',names='Sentiments')
        st.plotly_chart(fig)

# Word cloud
st.sidebar.subheader("Word Cloud")
word_sentiment = st.sidebar.radio("Which Sentiment to Display?", tuple(pd.unique(data["labels"])))
if st.sidebar.checkbox("Show", False, key="6"):
    st.subheader(f"Word Cloud for {word_sentiment.capitalize()} Sentiment")
    df = data[data["labels"]==word_sentiment]
    words = " ".join(data["tweet"])
    processed_words = " ".join([word for word in words.split() if "http" not in word and not word.startswith("@") and word != "RT"])
    wordcloud = WordCloud(stopwords=STOPWORDS, background_color="white", width=800, height=640).generate(processed_words)
    plt.imshow(wordcloud)
    plt.xticks([])
    plt.yticks([])
    st.pyplot()

if __name__=='__main__': 
    main()

Overwriting app.py


In [10]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-03-23 06:28:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.197.133.26, 3.223.239.191, 34.226.165.133, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.197.133.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  19.2MB/s    in 0.7s    

2021-03-23 06:28:52 (19.2 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [11]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [31]:
get_ipython().system_raw('./ngrok http 8501 &')

In [32]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://a4d5094e7edc.ngrok.io


In [33]:
!streamlit run /content/app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.27.206:8501

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

2021-03-23 06:36:09.252 NumExpr defaulting to 2 threads.
2021-03-23 06:36:44.262567: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-23 06:36:45.453366: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-23 06:36:45.454355: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic libr